In [180]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
folds = kf.split(range(len(y)))
S_train_i = np.zeros(len(y))
print len(y)
for j, (train_idx, test_idx) in enumerate(folds):
    print (train_idx, test_idx)

30396
(array([    1,     2,     3, ..., 30393, 30394, 30395]), array([    0,     5,    10, ..., 30385, 30388, 30392]))
(array([    0,     1,     2, ..., 30391, 30392, 30395]), array([    9,    16,    18, ..., 30382, 30393, 30394]))
(array([    0,     1,     2, ..., 30393, 30394, 30395]), array([    4,     6,     7, ..., 30377, 30384, 30389]))
(array([    0,     4,     5, ..., 30393, 30394, 30395]), array([    1,     2,     3, ..., 30378, 30390, 30391]))
(array([    0,     1,     2, ..., 30392, 30393, 30394]), array([   15,    20,    26, ..., 30386, 30387, 30395]))


In [155]:
ID = np.array([1,2,3])

In [154]:
df = pd.DataFrame({'id':[1,2], 'y':[4,5]})

In [157]:
df2 = pd.DataFrame(ID, columns=['id'])

In [158]:
df2.merge(df, on='id', how='left')

,id,y
0,1,4.0
1,2,5.0
2,3,NaN


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
from sklearn import ensemble   
from sklearn import datasets   
from sklearn.utils import shuffle   
import xgboost as xgb
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

In [2]:
class XgbModel(object):
    def __init__(self, params, num_boost_rounds):
        self.params = params
        self.num_boost_rounds = num_boost_rounds
        
    def copy(self):
        return XgbModel(self.params, self.num_boost_rounds)
    
    def fit(self, X, y):
        xgtrain = xgb.DMatrix(X, y)
        self.model = xgb.train(self.params, xgtrain, self.num_boost_rounds)
        
    def predict(self, X):
        xgtest = xgb.DMatrix(X)
        return self.model.predict(xgtest)

In [3]:
from stacking import Stacking

In [275]:
import stacking

In [276]:
reload(stacking)

<module 'stacking' from 'stacking.py'>

In [4]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [5]:
def logrmse(predictions, targets):
    return rmse(np.log(predictions), np.log(targets))

# bruno

In [6]:
import bruno

In [34]:
reload(bruno)

<module 'bruno' from 'bruno.py'>

In [45]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 420  # From Bruno's original CV, I think
bruno_xgb = XgbModel(xgb_params, num_boost_rounds)

# gunja

In [9]:
import gunja

In [10]:
reload(gunja)

<module 'gunja' from 'gunja.pyc'>

In [44]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 422
gunja_xgb = XgbModel(xgb_params, num_boost_rounds)

# louis

In [12]:
import louis

In [13]:
reload(louis)

<module 'louis' from 'louis.pyc'>

In [43]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 385  # This was the CV output, as earlier version shows
louis_xgb = XgbModel(xgb_params, num_boost_rounds)

# stacking

In [57]:
df_train = pd.read_csv('../input/train.csv')
id_train = df_train['id'].values
y = df_train['price_doc'].values

In [23]:
class DataResolver(object):
    def __init__(self):
        self.__time = -1
    
    def next(self):
        self.__time = self.__time + 1
        if (self.__time >= 2):
            return louis.resolver()
        if (self.__time >= 1):
            return gunja.resolver()
        return bruno.resolver()

In [47]:
base_models = []
base_models.append(bruno_xgb)
base_models.append(gunja_xgb)
base_models.append(louis_xgb)
stacking = Stacking(5, base_models, DataResolver(), logrmse)

In [48]:
s_train, s_test = stacking.fit(id_train, 7662)#38132 30471

(38132, 390)
(38132L, 394L)
(38132L, 394L)
model 0
train: 0.420613501748 val: 0.477737831655
train: 0.425007006396 val: 0.459305298253
train: 0.42039672604 val: 0.480549373028
train: 0.42860796861 val: 0.450614562747
train: 0.423228378048 val: 0.471364509042
model 1
train: 0.383467659704 val: 0.452678448878
train: 0.384376725088 val: 0.45505127241
train: 0.376932483776 val: 0.468695006951
train: 0.386184074091 val: 0.45296886144
train: 0.381288729246 val: 0.453990547096
model 2
train: 0.431167836295 val: 0.472524714846
train:

d:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app


 nan val: 0.453097897971
train: 0.427901431139 val: 0.483505017565
train: 0.435069040544 val: 0.458759965108
train: nan val: 0.476115483132


In [53]:
pd.DataFrame(s_train).to_csv('../stacking/5_3_train.csv', index=False)

In [54]:
pd.DataFrame(s_test).to_csv('../stacking/5_3_test.csv', index=False)

In [49]:
s_train

array([[  3804734.  ,   3631560.25,   5081369.  ],
       [  5614989.5 ,   4439626.5 ,   5466862.  ],
       [  4921093.  ,   4305226.5 ,   4645399.5 ],
       ..., 
       [  5405239.  ,   6122101.  ,   5312807.  ],
       [ 11368580.  ,  12399014.  ,  10650168.  ],
       [  6148419.  ,   6521823.5 ,   6236720.5 ]])

In [288]:
s_train

array([[  4285453.5 ,   4452850.  ,   4986183.5 ],
       [  4463656.5 ,   3639487.  ,   5244543.5 ],
       [  4424056.5 ,   4173486.25,   4886411.  ],
       ..., 
       [  5769501.  ,   5717440.  ,   5201236.5 ],
       [ 10980000.  ,  10370804.  ,  10273804.  ],
       [  6817400.  ,   7259843.5 ,   6272357.  ]])

In [50]:
y

NameError: name 'y' is not defined

In [51]:
s_test

array([[ 5409973.4 ,  5637508.3 ,  5347157.5 ],
       [ 8196299.7 ,  8380193.2 ,  8043320.5 ],
       [ 5342534.7 ,  5518024.  ,  5332160.5 ],
       ..., 
       [ 5580179.9 ,  4020587.4 ,  4784553.3 ],
       [ 5122271.1 ,  4301102.45,  5265858.6 ],
       [ 9594133.2 ,  8151672.7 ,  8513476.5 ]])

In [60]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(s_train, y)
dtest = xgb.DMatrix(s_test)


num_boost_rounds = 200
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds, evals={(dtrain,'train')})


y_predict = model.predict(dtest)

[0]	train-rmse:8.19075e+06
[1]	train-rmse:7.82501e+06
[2]	train-rmse:7.47812e+06
[3]	train-rmse:7.14982e+06
[4]	train-rmse:6.84017e+06
[5]	train-rmse:6.54405e+06
[6]	train-rmse:6.26787e+06
[7]	train-rmse:6.00661e+06
[8]	train-rmse:5.76052e+06
[9]	train-rmse:5.52528e+06
[10]	train-rmse:5.30721e+06
[11]	train-rmse:5.10404e+06
[12]	train-rmse:4.90834e+06
[13]	train-rmse:4.7232e+06
[14]	train-rmse:4.55258e+06
[15]	train-rmse:4.39164e+06
[16]	train-rmse:4.24178e+06
[17]	train-rmse:4.10192e+06
[18]	train-rmse:3.97146e+06
[19]	train-rmse:3.8475e+06
[20]	train-rmse:3.73522e+06
[21]	train-rmse:3.6265e+06
[22]	train-rmse:3.526e+06
[23]	train-rmse:3.43116e+06
[24]	train-rmse:3.34406e+06
[25]	train-rmse:3.26159e+06
[26]	train-rmse:3.1865e+06
[27]	train-rmse:3.1141e+06
[28]	train-rmse:3.04911e+06
[29]	train-rmse:2.98859e+06
[30]	train-rmse:2.93423e+06
[31]	train-rmse:2.88385e+06
[32]	train-rmse:2.83701e+06
[33]	train-rmse:2.79412e+06
[34]	train-rmse:2.75239e+06
[35]	train-rmse:2.7122e+06
[36]	train

In [61]:
df_test = pd.read_csv('../input/test.csv')
id_test = df_test.id

In [65]:
stacking_output = pd.DataFrame({'id': id_test, 'price_doc': y_predict * 0.97})

In [66]:
stacking_output.to_csv('../stacking.csv', index=False)

In [292]:

gunja_output = pd.DataFrame({'id': pd.read_csv('../input/test.csv').id, 'price_doc': y_predict}) 

In [293]:
gunja_output.to_csv('../stacking.csv', index=False)

In [286]:
id, X, y, T = gunja.resolver()

In [287]:
y.shape

(30396L,)

In [187]:
id, X, y, T = louis.resolver()

In [134]:
X, y, T = bruno.Resolver().get()

(38132, 391)
(38132L, 395L)
(38132L, 395L)


In [101]:
louis_xgb.fit(X, y)

In [102]:
y_pred = louis_xgb.predict(X)

In [103]:
logrmse(y, y_pred)

2.9603244826493116